In [1]:
import keras
import tensorflow as tf
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import random
import pandas as pd

In [8]:
max_sentence = 50000
# with open("text/shakespear.txt", "r") as file:
#     corpus = file.read()[max_sentence:max_sentence*2].lower()
convs = pd.read_csv("Conversation.csv")[:max_sentence]

In [9]:
corpus = ""
for t in convs["question"]:
    corpus += t + " "
for t in convs["answer"]:
    corpus += t + " "

In [10]:
# make cuts to generate data
num_examples = 500000
max_text_length = 100
x_train_text = []

for i in range(num_examples):
    start = i * max_text_length
    end = start + max_text_length
    if end > len(corpus):
        continue
    text = corpus[start:end]
    
    x_train_text.append(text)


# generate the one hot vectors for the characters

characters = set()
for c in corpus:
    characters.add(c)
    
    
len_characters = len(characters)

character_index = {}

for i, c in enumerate(characters):
    character_index[c] = i + 1

    
def character_to_one_hot(c):
    index = character_index.get(c)
    vec = np.array([(i + 1 == index) for i in range(len_characters)], dtype=np.float32)
    return vec

x_train = []

for text in x_train_text:
    x = np.array([character_to_one_hot(c) for c in text[:-1]])
    # if x.shape[0] > max_text_length:
    #     x = x[:max_text_length]
    # while x.shape[0] < max_text_length:
    #     # print("still small")
    #     x = np.vstack([x, np.array([0 for _ in range(len_characters)])])
    x_train.append(x)
x_train = np.array(x_train)

# y_train = np.array([character_to_one_hot(text[-1]) for text in x_train_text])
y_train = x_train[:, 1:, :]
x_train = x_train[:, :-1, :]

# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train)

In [ ]:
x_train_text

In [11]:
model = models.Sequential([
    layers.GRU(units=128, input_shape=(None, len_characters), return_sequences=True),
    layers.GRU(units=128, return_sequences=True),
    # layers.GRU(units=64, return_sequences=True),
    layers.Dense(units=len_characters, activation="softmax"),
])

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
 3/77 [>.............................] - ETA: 7:23 - loss: 3.7073 - accuracy: 0.1344

KeyboardInterrupt: 

In [12]:
sentence = "i"
length = 50
total_dashes = 20
for i in range(length):
    x_input = []
    sent = sentence
    if len(sentence) > max_text_length:
        sent = sentence[len(sent) - max_text_length:]
    for s in sent:
        ohv = character_to_one_hot(s)
        x_input.append(ohv)
    x_input = np.array(x_input)
    y_pred = model.predict(np.array([x_input]), verbose=None)[:, -1, :].reshape(-1, len_characters)
    c = tf.random.categorical(tf.math.log(y_pred), 1)
    c = c.numpy()[0] + 1
    for char in character_index.keys():
        if character_index[char] == c:
            c = char
            break
    sentence += c
    t = i/(length - 1)
    num_dashes = int(total_dashes * (1 - t))
    num_stars = int(total_dashes * t)
    print("\r" + ("."*num_stars) + ("-"*num_dashes), end="")
print("\r", sentence)

 iaw@fyneio eraynsrbp.choms$qfhu8i.uogf:dae qÂobteey
